In [435]:
import csv
import numpy as np
import copy
import math

In [376]:
def split_train_label(data):
    train_x = []
    train_y = []
    for i in data:
        train_x.append(i[1:])
        train_y.append([i[0]])
        
    return train_x,train_y

In [377]:
with open('titanic_data.csv','r') as file:
    temp = csv.reader(file)
    data = list(temp)

header = data[0]
data = data[1:]
for i in range(len(data)):
    row_len = len(data[0])
    for j in range(row_len):
        data[i][j] = float(data[i][j])
    
train_x, train_y = split_train_label(data)

In [378]:
header

['Survived',
 'Pclass',
 'Sex',
 'Age',
 'Siblings/Spouses Aboard',
 'Parents/Children Aboard',
 'Fare']

In [379]:
#normalization
for i in range(len(header[1:])):
    total = 0
    avg = 0
    if header[1:][i] not in ['Sex','Pclass']:
        for j in train_x:
            total += j[i]  
        avg = total/len(train_x)      
        for j in range(len(train_x)):
            train_x[j][i] = train_x[j][i]/avg

In [380]:
train_x

[[3.0, 0.0, 0.7464853332884488, 1.9034334763948497, 0.0, 0.22442054489674754],
 [1.0, 1.0, 1.2893837574982299, 1.9034334763948497, 0.0, 2.2065430383501137],
 [3.0, 1.0, 0.8822099393408941, 0.0, 0.0, 0.24531487149058265],
 [1.0, 1.0, 1.187590302958896, 1.9034334763948497, 0.0, 1.6436870253816958],
 [3.0, 0.0, 1.187590302958896, 0.0, 0.0, 0.2491841912301818],
 [3.0, 0.0, 0.9161410908540054, 0.0, 0.0, 0.26182293722760824],
 [1.0, 0.0, 1.8322821817080108, 0.0, 0.0, 1.6053807599596648],
 [3.0,
  0.0,
  0.06786230302622262,
  5.710300429184549,
  2.6088235294117643,
  0.6523673080964075],
 [3.0, 1.0, 0.9161410908540054, 0.0, 5.217647058823529, 0.3446263796550289],
 [2.0, 1.0, 0.47503612118355837, 1.9034334763948497, 0.0, 0.9308283202042918],
 [3.0,
  1.0,
  0.13572460605244524,
  1.9034334763948497,
  2.6088235294117643,
  0.5169411172104391],
 [1.0, 1.0, 1.9680067877604561, 0.0, 0.0, 0.8218435126908479],
 [3.0, 0.0, 0.6786230302622263, 0.0, 0.0, 0.2491841912301818],
 [3.0,
  0.0,
  1.323314

In [381]:
## encoding class

total_data = []
for i in train_x:
    temp = []
    if i[0] == 1:
        temp.append(1)
    else:
        temp.append(0)
    if i[0] == 2:
        temp.append(1)
    else:
        temp.append(0)
    if i[0] == 3:
        temp.append(1)
    else:
        temp.append(0)
    temp = temp + i[1:]
    total_data.append(temp)
train_x = total_data

In [382]:
## encoding class

train_x

[[0,
  0,
  1,
  0.0,
  0.7464853332884488,
  1.9034334763948497,
  0.0,
  0.22442054489674754],
 [1,
  0,
  0,
  1.0,
  1.2893837574982299,
  1.9034334763948497,
  0.0,
  2.2065430383501137],
 [0, 0, 1, 1.0, 0.8822099393408941, 0.0, 0.0, 0.24531487149058265],
 [1,
  0,
  0,
  1.0,
  1.187590302958896,
  1.9034334763948497,
  0.0,
  1.6436870253816958],
 [0, 0, 1, 0.0, 1.187590302958896, 0.0, 0.0, 0.2491841912301818],
 [0, 0, 1, 0.0, 0.9161410908540054, 0.0, 0.0, 0.26182293722760824],
 [1, 0, 0, 0.0, 1.8322821817080108, 0.0, 0.0, 1.6053807599596648],
 [0,
  0,
  1,
  0.0,
  0.06786230302622262,
  5.710300429184549,
  2.6088235294117643,
  0.6523673080964075],
 [0,
  0,
  1,
  1.0,
  0.9161410908540054,
  0.0,
  5.217647058823529,
  0.3446263796550289],
 [0,
  1,
  0,
  1.0,
  0.47503612118355837,
  1.9034334763948497,
  0.0,
  0.9308283202042918],
 [0,
  0,
  1,
  1.0,
  0.13572460605244524,
  1.9034334763948497,
  2.6088235294117643,
  0.5169411172104391],
 [1, 0, 0, 1.0, 1.9680067877

In [461]:
# each x data is a row vector. y is a column vector. Need header to do normalization

class logistic_titanic:
    def __init__(self, gradientRate= 3/4, max_iter = 2000, abstol = 1e-8, add_intercept = True):
        self.max_iter =  max_iter
        self.abstol = abstol
        #self.reltol = reltol
        self.add_intercept = add_intercept
        self.gradientRate = gradientRate
        
    def likelihood_score(self):
        likelihood = 0
        for i in range(len(self.training_y)):
            temp = 0
            x = np.array([self.training_x[i]]).T
            temp += self.training_y[i]*math.log(1/(1+ (math.exp((-np.dot(self.theta.T,x))))))
            temp += (1-self.training_y[i])*math.log((1/(1+ (math.exp((np.dot(self.theta.T,x)))))))
            likelihood += temp
        print(likelihood)
        return likelihood
    def gradient(self):
        gradient = np.zeros((len(self.training_x[0]),1))
        for i in range(len(self.training_y)):
            x = np.array([self.training_x[i]]).T
            temp = self.training_y[i] - (1/(1+ (math.exp((-np.dot(self.theta.T,x))))))
            gradient = gradient + temp*x
        
        return gradient
    
    def logistic_predict(self):
        answer = []
        for i in range(len(self.training_y)):
            temp = 0
            temp_x = np.array([self.training_x[i]]).T
            temp = 1/(1+math.exp(-np.dot(self.theta.T,temp_x)))
            if temp > 1/2:
                answer.append(1)
            else:
                answer.append(0)
        return answer
    def fit(self,x,y): 
        ## deep copy data
        self.training_x = np.array(copy.deepcopy(x))
        self.training_y = np.array(copy.deepcopy(y))
        print(self.training_x)
        
        ## add intercept
        data_num = len(self.training_x)
        if self. add_intercept == True:
            temp = []
            for i in range(data_num):
                temp.append([1])
            self.training_x = np.append(self.training_x,temp,axis = 1)

        ## initialize theta      
        theta = []
        
        for i in range(len(self.training_x[0])):
            theta.append([1])
        
        self.theta = np.array(theta)
            
        
        ## start training
        last_likelihood = float('-inf') 
        parameter_rate = 1/20
        for i in range(self.max_iter):
            if i % 100 == 0:
                parameter_rate = parameter_rate*self.gradientRate
            current_likelihood = self.likelihood_score()
            if abs(current_likelihood - last_likelihood) <= self.abstol:
                break
            last_likelihood = current_likelihood
            gradient_val = self.gradient()
            self.theta = self.theta + parameter_rate*gradient_val
            
            

        return(self.theta)

In [468]:
c = a.logistic_predict()

In [470]:
correct= 0
for i in range(len(train_y)):
    if train_y[i][0] == c[i]:
        correct +=1
print(correct/len(train_y))

0.8027057497181511


In [463]:
theta = a.fit(train_x, train_y)

[[0.         0.         1.         ... 1.90343348 0.         0.22442054]
 [1.         0.         0.         ... 1.90343348 0.         2.20654304]
 [0.         0.         1.         ... 0.         0.         0.24531487]
 ...
 [0.         0.         1.         ... 1.90343348 5.21764706 0.72588438]
 [1.         0.         0.         ... 0.         0.         0.92863674]
 [0.         0.         1.         ... 0.         0.         0.23989782]]
[-3169.30683935]
[-35158.30018324]
[-4530.46432389]
[-14176.53430531]
[-21169.34079253]
[-5470.19161184]
[-4701.91867623]
[-6927.47471316]
[-14131.90798838]
[-6435.85120689]
[-5385.20783956]
[-7468.08394948]
[-12342.29094455]
[-5974.17703189]
[-5282.08430984]
[-6668.96621794]
[-11408.98785918]
[-5284.93281481]
[-4940.68282759]
[-6487.64152766]
[-11398.38681909]
[-5357.70698787]
[-5193.10422347]
[-6553.74491828]
[-11108.94742847]
[-5278.06055941]
[-5065.16255507]
[-6113.91274523]
[-9703.49595575]
[-5655.12381746]
[-6634.14680422]
[-6638.20995555]
[-98

[-2182.20176143]
[-1900.91473512]
[-2180.11903982]
[-1901.22725243]
[-2182.38551969]
[-1902.90289623]
[-2183.51896174]
[-1903.43688857]
[-2182.89037927]
[-1903.85931711]
[-2183.89001848]
[-1904.88705694]
[-2184.75209876]
[-1905.52791081]
[-2184.76748703]
[-1906.01838565]
[-2185.31278422]
[-1906.73997168]
[-2185.92441711]
[-1907.34041224]
[-2186.17232275]
[-1907.83894406]
[-2186.55960443]
[-1908.40824682]
[-2187.00828608]
[-1908.93783358]
[-2187.31191309]
[-1909.40858519]
[-2187.63531475]
[-1909.88992051]
[-2187.98722123]
[-1910.35262688]
[-2188.28229542]
[-1910.78134569]
[-2188.56810172]
[-1911.20142559]
[-2188.8607523]
[-1911.60742179]
[-2189.12905356]
[-1911.99172233]
[-2189.38456634]
[-1192.07938425]
[-1093.87523297]
[-941.16931779]
[-947.19137068]
[-892.22507418]
[-963.89935113]
[-1023.85199835]
[-1336.85041069]
[-1555.30988949]
[-2219.03927695]
[-1375.09821766]
[-1298.64240826]
[-1207.05478978]
[-1342.16964452]
[-1393.1741739]
[-1815.45913128]
[-1463.61170743]
[-1536.59440274]
[-1

[-504.55733952]
[-536.90426462]
[-504.68728204]
[-537.0723803]
[-504.80907458]
[-537.22865953]
[-504.92384711]
[-537.3744103]
[-505.03216557]
[-537.51043414]
[-505.13448943]
[-537.63760182]
[-505.23138326]
[-537.75687144]
[-505.32340351]
[-537.86906991]
[-505.41098552]
[-537.97483453]
[-505.49446707]
[-538.07470318]
[-505.57415288]
[-538.16918048]
[-505.65033257]
[-538.25873132]
[-505.72326541]
[-538.34376037]
[-505.79317354]
[-538.4246167]
[-505.86025104]
[-538.50161095]
[-505.9246737]
[-538.57502485]
[-505.98660162]
[-538.64511205]
[-506.04617884]
[-538.71209888]
[-506.10353459]
[-538.77618814]
[-427.92021226]
[-416.13311868]
[-406.8806626]
[-404.31342655]
[-402.08449331]
[-401.18347853]
[-400.18548999]
[-399.73770384]
[-399.11496038]
[-398.90203146]
[-398.47854039]
[-398.49085464]
[-398.22909166]
[-398.53467632]
[-398.43824191]
[-399.18233097]
[-399.26706879]
[-400.70183155]
[-400.95691436]
[-403.46251564]
[-403.76103021]
[-407.78635427]
[-407.73759315]
[-413.56792631]
[-412.4367962

[-390.46506725]
[-390.46505689]
[-390.46504664]
[-390.4650365]
[-390.46502647]
[-390.46501654]
[-390.46500671]
[-390.46499699]
[-390.46498737]
[-390.46497785]
[-390.46496843]
[-390.4649591]
[-390.46494988]
[-390.46494075]
[-390.46493172]
[-390.46492278]
[-390.46491614]
[-390.46490956]
[-390.46490302]
[-390.46489654]
[-390.46489011]
[-390.46488372]
[-390.46487739]
[-390.46487111]
[-390.46486488]
[-390.46485869]
[-390.46485256]
[-390.46484647]
[-390.46484043]
[-390.46483444]
[-390.4648285]
[-390.4648226]
[-390.46481675]
[-390.46481094]
[-390.46480518]
[-390.46479947]
[-390.4647938]
[-390.46478817]
[-390.46478259]
[-390.46477705]
[-390.46477156]
[-390.46476611]
[-390.4647607]
[-390.46475533]
[-390.46475001]
[-390.46474473]
[-390.46473949]
[-390.46473429]
[-390.46472913]
[-390.46472401]
[-390.46471894]
[-390.4647139]
[-390.4647089]
[-390.46470394]
[-390.46469902]
[-390.46469414]
[-390.4646893]
[-390.46468449]
[-390.46467973]
[-390.464675]
[-390.4646703]
[-390.46466565]
[-390.46466103]
[-39

In [469]:
c

[0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,


In [455]:
def logistic_predict(data,theta):
    answer = []
    for i in data:
        temp = 0
        temp_x = np.array([i]).T
        temp = 1/(1+math.exp(-np.dot(theta.T,temp_x)))
        if temp > 1/2:
            answer.append(1)
        else:
            answer.append(0)

In [457]:
logistic_predict(train_x,theta)

ValueError: shapes (1,9) and (8,1) not aligned: 9 (dim 1) != 8 (dim 0)

In [149]:
a= np.array([[1,2,3],[4,5,6]])

In [122]:
x

array([ -2.   ,  -0.5  , -11.5  ,  -1.   ,  -0.5  ,  -4.125])

In [175]:
np.append(a,[[1],[1]],axis = 1)

ValueError: zero-dimensional arrays cannot be concatenated

In [240]:
float('-inf') 

-inf

In [297]:
np.exp(100)

2.6881171418161356e+43